In [2]:
import numpy as np
#from imageio import imread, imwrite
import cv2

## Preparation


In [3]:
grainger_image = cv2.imread('grainger.jpg')

## Naive Method: Increase blue channel

In [4]:
arr = grainger_image * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
#window_name = 'image'
#cv2.imshow(window_name, gamma_img)
cv2.imwrite("night_final.png", gamma_img)

True

## Intermediate Method

In [53]:
def turn_day_to_night(image):
    '''
    Intermediate method looks to increase the contrast of an image
    Decrease brightness
    Decrease saturation
    Increase blue-tint of image
    Add noise to image
    Increase contrast
    '''
    image_width, image_height, color_channels = image.shape

    # Convert image to YUV color space and decrease brightness by half
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    yuv_image[:,:,0] = np.clip(yuv_image[:,:,0] * 0.5, 0, 255) 
    bgr_image = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)

    
    # Convert image to HSV color space and decrease warmth
    bgr_image[:,:,0] = bgr_image[:,:,0] + 30
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 1] = hsv_image[:, :, 1] * 0.8
    bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    
    # Adjust contrast
    bgr_image = cv2.convertScaleAbs(bgr_image, alpha=0.8, beta=0)

    #Adding noise to image
    noise_filter = np.zeros((image_width, image_height, color_channels), np.uint8)
    cv2.randu(noise_filter,(0),(10))
    
    result_image = bgr_image + noise_filter

    return result_image

In [54]:
night_image = turn_day_to_night(grainger_image)
cv2.imwrite("night_final.png", night_image)

True